In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate

# ¿CUÁL ES EL VECINO MAS CERCANO EN EL SISTEMA SOLAR?

¿Cuál es el planeta mas cercano a la tierra? La respuesta a esta pregunta parece obvia, es intuitivo pensar en alguno de los dos vecinos inmediatos, venus o marte; pero realmente son las **órbitas** de estos planetas las que son cercanas a la **órbita** de la tierra. Como se puede observar en la imagen inferior, hay un orden definido para las órbitas en el sistema solar, órbitas que son elípticas según la Primera ley de Kepler por lo que su posición desde el sol está definida por su semieje mayor; una consecuencia de tener órbitas elípticas es que la distancia del planeta al sol no es constante durante su trayectoria. El problema de este razonamiento es que no se considera la velocidad relativa entre los planetas ni el hecho de que, según la Tercera ley de Kepler el período de rotación de un planeta depende del semieje mayor de su órbita de acuerdo con la siguente relación:

$$\frac{T^2}{a3} = k$$

Donde $T$ es el período orbital, $a$ es el seimeje mayor de su órbita y $k$ es la consante de Kepler. De esta ley se puede concluir que mientras mas alejado esté un planeta de su estrella, mas tiempo tarda en dar una vuelta completa a su órbita; de esto se deriva que la rapidez angular $\omega$ de cada planeta es distinta y, finalmente, que la distancia entre órbitas no indique la distancia entre planetas en algún momento dado.

![Tomado de: https://significado.com/sistema-solar/](https://significado.com/img/ciencia/Sistema-Solar-planetas-sol.jpg)

Esta pregunta inspiró el video [¿Cual planeta es el mas cercano?](https://www.youtube.com/watch?v=SumDHcnCRuU&ab_channel=CGPGrey) del canal *CGP Grey* donde se ahonda aún más en esta cuestión ampliándola a todo el sistema solar, cambiando así la pregunta a ¿Cuál es el planeta mas cercano a cada planeta del sistema solar? Este video sirvió de inspiración y base para este proyecto.

En la tabla a continuación se muestra el valor del semieje mayor de cada planeta; estos números fueron tomados de una base de datos de [Princeton]( https://www.princeton.edu/~willman/planetary_systems/Sol/) con información sobre el sistema solar. Para este análisis resulta acertado tomar las órbitas del sistema solar como circunferencias, debido a que la excentricidad promedio de cada órbita es muy pequeña: de 0.06; por lo que en vez de semieje mayor se puede hablar de radio orbital.

In [2]:
planetas=['Mercurio','Venus','Tierra','Marte','Jupiter','Saturno','Urano','Neptuno'] #lista con nombre de los planetas
a=np.array([0.3870993,0.723336,1.000003,1.52371,5.2029,9.537,19.189,30.0699]) # Array con semiejes mayores en UA

dfp=pd.DataFrame({'Planeta': planetas,'Radio orbital':a}) #DataFrame con el semieje maot de cada planeta

dfp

,Planeta,Radio orbital
0,Mercurio,0.387099
1,Venus,0.723336
2,Tierra,1.000003
3,Marte,1.523710
4,Jupiter,5.202900
5,Saturno,9.537000
6,Urano,19.189000
7,Neptuno,30.069900


Pensando en como responder a la pregunta antes formulada, se desarrolló el método de punto-círculo, point-circle method en inglés o *PCM*, el cual es explicado en el artículo ['Venus no es el vecino mas cercano a la tierra'](https://physicstoday.scitation.org/do/10.1063/PT.6.3.20190312a/full/) de *Physics today*. Este método permite calcular la distancia promedio entre dos planetas utilizando una integral elíptica de segunda especie que es mostrada a continuación:

$$\bar{d}=\frac{2}{\pi}(r_1+r_2)E\left(\frac{2\sqrt{r_1r_2}}{r_1+r_2}\right)$$

$$E(k)=\int_0^{\pi/2}{\sqrt{1-k^2\sin^2{\theta}}}\mathrm{d}\theta$$

Donde $\bar{d}$ es la distancia promedio y, $r_1$ y $r_2$ son los radios orbitales en unidades astronómicas.

En el atículo se explica como, por medio de una simulación se encontró que el antiguo método para calcular la distancia entre planetas presentaba varias fallas mientras que, con el *PCM* se pueden hallar resultados bastante precisos.

Con esto en mente, el proyecto se basa en utilizar este método el cuál, gracias a los conocimientos adquiridos en integración numérica puede implementarse de manera sencilla; solo hay que resolver la integral elíptica e implementar la fórmula de la distancia promedio para resolver esta interrogante.

In [3]:
# Se calcula la integral elíptica
# Se definen las funciones necesarias

# k es el término que define la integral elíptica
def k(r1,r2): 
  return ((2*np.sqrt(r1*r2))/(r1+r2))


# Integral elíptica de segunda especie
def E(k):
  xe=np.linspace(.00001,np.pi/2-.00001,1000)
  e = lambda θ: np.sqrt(1-k**2*np.sin(θ)**2)
  return integrate.simps(e(xe),xe)

# Valor de la distancia promedio
def dp(E,r1,r2):
  return 2/np.pi*(r1+r2)*E

Para el código solo se hizo uso del módulo integrate de la ibrería scipy para resolver la integral elíptica; para la tabla de distancias se utilizó la librería pandas.

In [5]:
dpl = []
for i1 in range(len(a)):
  r1 = a[i1]
  for i2 in range(len(a)):
    r2 = a[i2]
    if r2 == r1:
      dpl.append(0)
    else:
      k1=k(r1,r2)
      E1=E(k1)
      dp1=dp(E1,r1,r2)
      dpl.append(dp1)

dm = dpl[:8] # Distancias de mercurio
dv = dpl[8:16] # Distancias de venus
de = dpl[16:24] # Distancias de la tierra
dma = dpl[24:32] # Distancias de marte
dj = dpl[32:40] # Distancias de jupiter
ds = dpl[40:48] # Distancias saturno
du = dpl[48:56] # Distancias de urano
dn = dpl[56:64] # Distancias de neptuno

dfd = pd.DataFrame({'Distancia a':planetas,
                    'Mercurio':dm,
                    'Venus':dv,
                    'Tierra':de,
                    'Marte':dma,
                    'Jupiter':dj,
                    'Saturno':ds,
                    'Urano':du,
                    'Neptuno':dn})
dfd

,Distancia a,Mercurio,Venus,Tierra,Marte,Jupiter,Saturno,Urano,Neptuno
0,Mercurio,0.000000,0.776119,1.037816,1.548377,5.210036,9.540807,19.190708,30.070763
1,Venus,0.776119,0.000000,1.135787,1.610820,5.228005,9.550599,19.195573,30.073867
2,Tierra,1.037816,1.135787,0.000000,1.692760,5.250996,9.563110,19.201786,30.077832
3,Marte,1.548377,1.610820,1.692760,0.000000,5.315003,9.597836,19.219015,30.088823
4,Jupiter,5.210036,5.228005,5.250996,5.315003,0.000000,10.260804,19.543085,30.295002
5,Saturno,9.540807,9.550599,9.563110,9.597836,10.260804,0.000000,20.393284,30.830587
6,Urano,19.190708,19.195573,19.201786,19.219015,19.543085,20.393284,0.000000,33.218306
7,Neptuno,30.070763,30.073867,30.077832,30.088823,30.295002,30.830587,33.218306,0.000000


En conclusión, sin importar el planeta del que se esté hablando, mercurio simepre será el *vecino* mas cercano la mayor parte del tiempo; esto fue provado por la simulación de la que se habla en el artículo pero se pueden obtener resultados precisos que avalen esto por medio del **PCM**.

## Referencias

- Stockman, T., Monroe, G. and Cordner, S. (2019) “Venus is not earth’s closest neighbor,” Physics Today [Preprint]. Available at: https://doi.org/10.1063/pt.6.3.20190312a. 

- Willman, A. (2021) “Sol Planetary System Data.” Available at: https://www.princeton.edu/~willman/planetary_systems/Sol/ (Accessed: November 15, 2021). 

- Which Planet is the Closest? (2019) YouTube. CGP Grey. Available at: https://www.youtube.com/watch?v=SumDHcnCRuU&amp;ab_channel=CGPGrey (Accessed: November 12, 2022). 


